# Liability-driven Investing
_The dominant method in modern investing_

Investing has two goals, delivering performance and protecting downside which can be schizophrenic goals. Instead of building a single portfolio to address both of these goals two portfolios are created:
* Performance-seeking portfolio (PSP)
    * Focus on diversified, efficient access to risk premia
* Liability-hedging portfolio (LHP)
    * Focus on hedging impacts of risk factors in liabilities
    
Just as in options pricing, LDI has greeks:
* $\lambda_{psp}$ is the PSP Sharpe ratio: $\lambda=0 \Rightarrow$ no investment in PSP
* $\beta_{lhp}$ is the beta of liabilities with regard to LHP: $\beta=0 \Rightarrow$ no investment in LHP
* $\sigma_{psp}$ is the PSP volatility: $\sigma=\infty \Rightarrow$ no investment in PSP
* $\gamma$ is the risk-aversion_ $\gamma=\infty \Rightarrow$ no investment in PSP
